# DeepLearning - MNIST

### Tensorflow

---

In [ ]:
# pip uninstall tensorflow
# pip install tensorflow==1.14

In [ ]:
import tensorflow as tf

### 데이터 다운로드 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 1. 인공신경망

In [ ]:
# Session 생성
sess = tf.InteractiveSession()

In [ ]:
# 상수 영역 할당
x  = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
# 변수 영역 할당
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
# 초기화
sess.run(tf.global_variables_initializer())

In [ ]:
# 회귀모델 적용
y = tf.matmul(x,W) + b

In [ ]:
# 손실함수 교차엔트로피 사용
cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

In [ ]:
# 학습 모델(경사하강법) 생성
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
# 학습 수행
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [ ]:
# 결과 평가
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

In [ ]:
# Session 종료
sess.close()

# 2. 인공신경망 - MLP

### 2.1 함수 정의

In [ ]:
# Weight 초기값 설정 함수
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# Bias 초기값 설정 함수
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#
# stride = 0, zero 패딩, 2x2 max 풀링
#

# Convolution 정의 함수
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# Pooling 정의 함수
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### 2.2 Layer 설정

In [ ]:
# 1st Layer 파라미터 정의
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# 입력 데이터 재구성
x_image = tf.reshape(x, [-1, 28, 28, 1])

# 계수를 곱하고, 바이어스 더하고, 활성화 함수를 적용하고, 맥스 풀링 
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
# 2nd Layer 파라미터 정의
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 맥스 풀링: 이미지 크기 --> 7x7

# 전체연결망 (Fully Connected, FC)
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [ ]:
# 드롭아웃
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 출력층
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### 2.3 학습

In [ ]:
# 학습 횟수 설정
learn_cnt = 1000

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(learn_cnt):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

---

In [ ]:
# end of file